In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import os

In [ ]:
# Parameters and File Paths

# Parameters for data
WINDOW = 21     # rolling window size to use as predictors
DATE_COL = 'Date'
ID_COL = 'PERMNO'
TARGET_COL = 'excess_return'
WEIGHTS_COL = 'avg_market_cap'

# File path for the equal weighted portfolio results
current_directory = os.getcwd()
bu_equal_portfolio_results_path = os.path.join(current_directory, 'Results', f'bu_equal_portfolio_results{WINDOW:.0f}.csv')
ml_equal_portfolio_results_path = os.path.join(current_directory, 'Results', f'ml_equal_portfolio_results{WINDOW:.0f}.csv')

# File path for the value weighted portfolio results
bu_value_portfolio_results_path = os.path.join(current_directory, 'Results', f'bu_value_portfolio_results{WINDOW:.0f}.csv')
ml_value_portfolio_results_path = os.path.join(current_directory, 'Results', f'ml_value_portfolio_results{WINDOW:.0f}.csv')

# File path to save the equal weighted portfolio evaluation results
bu_equal_portfolio_evaluation_path = os.path.join(current_directory, 'Results', f'bu_equal_portfolio_evaluation{WINDOW:.0f}.csv')
ml_equal_portfolio_evaluation_path = os.path.join(current_directory, 'Results', f'ml_equal_portfolio_evaluation{WINDOW:.0f}.csv')

# File path to save the value weighted portfolio evaluation results
bu_value_portfolio_evaluation_path = os.path.join(current_directory, 'Results', f'bu_value_portfolio_evaluation{WINDOW:.0f}.csv')
ml_value_portfolio_evaluation_path = os.path.join(current_directory, 'Results', f'ml_value_portfolio_evaluation{WINDOW:.0f}.csv')

In [ ]:
# Dictionary for model names
models_dict = {"ols": "OLS",
               "lasso": "Lasso",
               "ridge": "Ridge",
               "enet": "Elastic Net",
               "rf": "RF",
               "xgb": "XGB",
               "nn1": "NN1",
               "nn2": "NN2",
               "nn3": "NN3",
               "nn4": "NN4",
               "nn5": "NN5",
               "tfm1": "TimesFM 1.0",
               "tfm2": "TimesFM 2.0",
               "chr_bolt_tiny": "Chronos-Bolt-Tiny",
               "chr_bolt_mini": "Chronos-Bolt-Mini",
               "chr_bolt_small": "Chronos-Bolt-Small",
               "chr_bolt_base": "Chronos-Bolt-Base",
               "chr_t5_tiny": "Chronos-T5-Tiny",
               "chr_t5_mini": "Chronos-T5-Mini",
               "chr_t5_small": "Chronos-T5-Small",
               "moirai_s": "Moirai-Small",
               "moirai_moe_s": "Moirai-MoE-Small",
               "moirai_moe_b": "Moirai-MoE-Base"
               }

### Step 1: Load Bottom-up and Long-Short Portfolios

In [ ]:
# Load the results for equal and value weighted bottom-up portfolios
bu_equal_portfolio_rets = pd.read_csv(bu_equal_portfolio_results_path, parse_dates=[DATE_COL])
bu_value_portfolio_rets = pd.read_csv(bu_value_portfolio_results_path, parse_dates=[DATE_COL])

# Load the results for equal and value weighted long-short portfolios
ml_equal_portfolio_rets = pd.read_csv(ml_equal_portfolio_results_path, parse_dates=[DATE_COL])
ml_value_portfolio_rets = pd.read_csv(ml_value_portfolio_results_path, parse_dates=[DATE_COL])

### Step 2: Economic Evaluation and Comparison of Bottom-up Portfolios

In [ ]:
# Function to Calculate Predictive-R2 Used in the Finance Literature
def r2(y_true, y_pred):
    return 1-(((y_true-y_pred)**2).sum()/(y_true**2).sum())

In [ ]:
# Evaluate Equal Weighted Bottom-up Portfolio Using Various Models
bu_equal_portfolio_r2 = []

models = [col for col in bu_equal_portfolio_rets.columns.to_list() if col.startswith("e_")]

for model in models:
    bu_equal_portfolio_r2.append({
        "Model": models_dict[model[2:]],
        "R2 (Equal)": r2(bu_equal_portfolio_rets["r_bu_portfolio"], bu_equal_portfolio_rets[model]),
        "DA (Equal)": (np.sign(bu_equal_portfolio_rets["r_bu_portfolio"]) == np.sign(bu_equal_portfolio_rets[model])).mean()
        })

bu_equal_portfolio_r2_df = pd.DataFrame(bu_equal_portfolio_r2)
bu_equal_portfolio_r2_df["DA (Equal)"] = bu_equal_portfolio_r2_df["DA (Equal)"]*100

In [ ]:
# Evaluate Value Weighted Bottom-up Portfolio Using Various Models
bu_value_portfolio_r2 = []

models = [col for col in bu_value_portfolio_rets.columns.to_list() if col.startswith("e_")]

for model in models:
    bu_value_portfolio_r2.append({
        "Model": models_dict[model[2:]],
        "R2 (Value)": r2(bu_value_portfolio_rets["r_bu_portfolio"], bu_value_portfolio_rets[model]),
        "DA (Value)": (np.sign(bu_value_portfolio_rets["r_bu_portfolio"]) == np.sign(bu_value_portfolio_rets[model])).mean()
        })

bu_value_portfolio_r2_df = pd.DataFrame(bu_value_portfolio_r2)
bu_value_portfolio_r2_df["DA (Value)"] = bu_value_portfolio_r2_df["DA (Value)"]*100

##### Save Results

In [ ]:
# Save Equal and Value Weighted Bottom-up Portfolio Evaluation Results
bu_equal_portfolio_r2_df.to_csv(bu_equal_portfolio_evaluation_path, index=False)
bu_value_portfolio_r2_df.to_csv(bu_value_portfolio_evaluation_path, index=False)

In [ ]:
bu_portfolio_r2_df = bu_equal_portfolio_r2_df.merge(bu_value_portfolio_r2_df, 'left', "Model")
bu_portfolio_r2_df

### Step 3: Evaluate Yearly Returns for Bottom-Up and Long-Short Portfolios

In [ ]:
# Function to Evaluate Realized Annual Returns
def annual_rets(df, date_col, r_ret_col):
    df = df.copy()
    df["Year"] = df[date_col].dt.year
    
    def a_annualize(group):
        return group[r_ret_col].sum()   # annual return

    a_ret = (df.groupby(["Year"]).apply(a_annualize)).reset_index().rename(columns={0:r_ret_col+"_p_annual"})
    return a_ret

In [ ]:
#  Calculate Realized Annual Returns for Bottom-Up Equal and Value Weighted Portfolio for Various Models
bu_equal_annual_rets_df = annual_rets(bu_equal_portfolio_rets, DATE_COL, "r_bu_portfolio")

bu_value_annual_rets_df = annual_rets(bu_value_portfolio_rets, DATE_COL, "r_bu_portfolio")
bu_equal_annual_rets_df.info()

In [ ]:
#  Calculate Realized Annual Returns for ML-Based Long-Short Equal Weighted Portfolio for Various Models
equal_annual_rets_df = bu_equal_annual_rets_df.copy()

for model in models:
    r_ret = "r_"+model[2:]
    a_ret = annual_rets(ml_equal_portfolio_rets, DATE_COL, r_ret)

    equal_annual_rets_df = equal_annual_rets_df.merge(a_ret, 'left', "Year")

equal_annual_rets_df = equal_annual_rets_df.set_index("Year").T
year_cols = equal_annual_rets_df.columns.to_list()
equal_annual_rets_df["Average"] = np.mean([equal_annual_rets_df[col] for col in year_cols], axis=0)
equal_annual_rets_df

In [ ]:
#  Calculate Realized Annual Returns for ML-Based Long-Short Value Weighted Portfolio for Various Models
value_annual_rets_df = bu_value_annual_rets_df.copy()

for model in models:
    r_ret = "r_"+model[2:]
    a_ret = annual_rets(ml_value_portfolio_rets, DATE_COL, r_ret)

    value_annual_rets_df = value_annual_rets_df.merge(a_ret, 'left', "Year")

value_annual_rets_df = value_annual_rets_df.set_index("Year").T
year_cols = value_annual_rets_df.columns.to_list()
value_annual_rets_df["Average"] = np.mean([value_annual_rets_df[col] for col in year_cols], axis=0)
value_annual_rets_df

### Step 4: Economic Evaluation and Comparison of Long-Short Portfolios

In [ ]:
# Function to Evaluate Sharpe Ratio, Maximum Drawdown and CAGR for Portfolio
def economic_evaluation(df, r_ret_col, date_col):
    df = df.copy()
    df["Year"] = df[date_col].dt.year

    mean_return = df[r_ret_col].mean()
    std_return = df[r_ret_col].std()
    sharpe_ratio = (mean_return / std_return) * ((252)**0.5)

    def max_dd(group):
        cumulative = (1 + group[r_ret_col]).cumprod()
        running_max = cumulative.cummax()
        drawdown = 1 - (cumulative / running_max)
        return drawdown.max()
    
    max_drawdown = (df.groupby(["Year"]).apply(max_dd)).reset_index()[0].max()

    cagr = ((np.prod([(1 + r) for r in df[r_ret_col]]))**(252/df.shape[0]) - 1)

    return mean_return, std_return, sharpe_ratio, max_drawdown, cagr

In [ ]:
#  Economic Evaluation of ML-Based Equal Weighted Portfolio for Various Models
ml_equal_portfolio_ee = []

for model in models:
    r_ret = "r_"+model[2:]
    a_ret = annual_rets(ml_equal_portfolio_rets, DATE_COL, r_ret)
    avg_annual_ret = np.mean(a_ret[r_ret+"_p_annual"])
    avg_daily_ret, std_daily_ret, sharpe_ratio, max_drawdown, cagr = economic_evaluation(ml_equal_portfolio_rets, r_ret, DATE_COL)

    ml_equal_portfolio_ee.append({
        "Model": models_dict[model[2:]],
        "Avg. Annual Return": avg_annual_ret,
        "Std. Annual Return": std_daily_ret * ((252)**0.5),
        "Sharpe Ratio": sharpe_ratio,
        "Maximum Drawdown": max_drawdown,
        "CAGR": cagr
        })
    
ml_equal_portfolio_ee_df = pd.DataFrame(ml_equal_portfolio_ee)
ml_equal_portfolio_ee_df["Avg. Annual Return"] = ml_equal_portfolio_ee_df["Avg. Annual Return"]*100
ml_equal_portfolio_ee_df["Std. Annual Return"] = ml_equal_portfolio_ee_df["Std. Annual Return"]*100
ml_equal_portfolio_ee_df["Maximum Drawdown"] = ml_equal_portfolio_ee_df["Maximum Drawdown"]*100
ml_equal_portfolio_ee_df["CAGR"] = ml_equal_portfolio_ee_df["CAGR"]*100

ml_equal_portfolio_ee_df

In [ ]:
#  Economic Evaluation of ML-Based Value Weighted Portfolio for Various Models
ml_value_portfolio_ee = []

for model in models:
    r_ret = "r_"+model[2:]
    a_ret = annual_rets(ml_value_portfolio_rets, DATE_COL, r_ret)
    avg_annual_ret = np.mean(a_ret[r_ret+"_p_annual"])
    avg_daily_ret, std_daily_ret, sharpe_ratio, max_drawdown, cagr = economic_evaluation(ml_value_portfolio_rets, r_ret, DATE_COL)

    ml_value_portfolio_ee.append({
        "Model": models_dict[model[2:]],
        "Avg. Annual Return": avg_annual_ret,
        "Std. Annual Return": std_daily_ret  * ((252)**0.5),
        "Sharpe Ratio": sharpe_ratio,
        "Maximum Drawdown": max_drawdown,        
        "CAGR": cagr
        })
    
ml_value_portfolio_ee_df = pd.DataFrame(ml_value_portfolio_ee)
ml_value_portfolio_ee_df["Avg. Annual Return"] = ml_value_portfolio_ee_df["Avg. Annual Return"]*100
ml_value_portfolio_ee_df["Std. Annual Return"] = ml_value_portfolio_ee_df["Std. Annual Return"]*100
ml_value_portfolio_ee_df["Maximum Drawdown"] = ml_value_portfolio_ee_df["Maximum Drawdown"]*100
ml_value_portfolio_ee_df["CAGR"] = ml_value_portfolio_ee_df["CAGR"]*100

ml_value_portfolio_ee_df

##### Save Results

In [ ]:
# Save Equal and Value Weighted ML-Based Portfolio Evaluation Results
ml_equal_portfolio_ee_df.to_csv(ml_equal_portfolio_evaluation_path, index=False)
ml_value_portfolio_ee_df.to_csv(ml_value_portfolio_evaluation_path, index=False)